# 第十二章 文件系统

## 12.1 基本概念

### 12.1.1 文件系统和文件

**文件系统**：一种用于持久性储存的系统抽象，它在储存器上组织、控制、导航、访问和检索数据。

**文件**：文件系统中一个单元的相关数据在操作系统中的抽象。

**文件属性**：名称、类型、大小、保护、创建者、创建时间、最近修改时间等。

**文件头**：
* 在储存元数据中保存了每个文件的信息。
* 保存文件的属性。
* 跟踪哪一块存储属于逻辑上文件结构的哪个偏移。

### 12.1.2 文件描述符

**文件描述符**：本质上是一个数字，内核跟踪每个进程打开的文件，并且操作系统为每个进程维护一个打开文件表，而一个打开文件描述符是这个表中的索引。

**管理打开文件的元数据**：
* 文件指针：指向最近一次的读写位置，每个打开了这个文件的进程都有这个指针。
* 文件打开计数：记录文件打开的次数 - 当最后一个进程关闭文件时，允许将其从打开文件表中移除。
* 文件磁盘位置：缓存数据访问信息。
* 访问权限：每个程序访问模式信息。

在文件系统中的所有操作都是在整个块空间上进行的：
* 磁盘管理空间是按照块区分的。
* 即使只对文件进行了一个字节的读写操作，文件系统也会读取和写回对应的块，而不是一个字节。

**用户访问文件**：
* 顺序访问：按字节读取。
* 随机访问：中中间读取。
* 基于内容访问：通过特征。


### 12.1.3 目录

**目录**：文件以目录的方式组织起来，目录是一类特殊的文件，每个目录包含一张表用于指示它所包含的文件。

**典型操作**：搜索文件、创建文件、删除文件、枚举目录、重命名文件。

**名字解析**：逻辑名字转换成物理资源的过程。

**挂载点**：
* 一个文件系统需要先挂载才能被访问。
* 一个未挂载的文件系统被挂载在挂载点上。

### 12.1.4 文件别名

两个或多个文件名关联同一个文件。
* 硬连接：多个文件项指向一个文件。
* 软连接：以“快捷方式”指向其他文件，”快捷方式“保存了真正文件的路径。

**删除一个有别名的文件**
* 硬连接中有一个计数器，记录了连接的个数，只有连接个数为零时删除文件。
* 软连接的“快捷方式”则会被为一个悬空指针。

### 12.1.5 文件系统种类

* 磁盘文件系统：文件存储在数据存储设备上，如磁盘。
* 数据库文件系统：文件根据其特征是可被寻址的，如WinFS。
* 日志文件系统：记录文件系统的修改/事件。
* 网络/分布式文件系统：文件可以通过网络被共享。
* 特殊/虚拟文件系统：存在在内存中的概念，用来对不同文件系统进行抽象。

## 12.2 虚拟文件系统

**目标**：对不同文件系统的抽象。

**功能**：
* 提供相同的文件和文件系统接口。
* 管理所有文件和文件系统关联的数据结构。
* 高效查询历程，遍历文件系统。
* 与特定文件系统模块的交互。

**卷控制块（Superblock）**：
* 每个文件系统一个。
* 文件系统详细信息。
* 块、块大小、空余块、计数/指针等。

**文件控制块（vnode、inode）**：
* 每个文件一个。
* 文件详细信息。
* 许可、拥有者、大小、数据库位置等。

**目录节点（dentry）**：
* 每个目录项一个。
* 将目录项数据结构及树型布局编码成树型数据结构。
* 指向文件控制块、父节点、项目列表等。

**加载时机**：
* 卷控制块：当文件系统挂载时进入内存。
* 文件控制块：当文件被访问时进入内存。
* 目录节点：在遍历一个文件路径时进入内存。

## 12.3 数据缓存

按需读/写，或者是预先/延迟读/写。

两种数据块缓存：普通缓冲区和页缓存（统一缓存数据块和内存页）

## 12.4 打开文件的数据结构

**打开文件描述**：
* 每个被打开的文件一个。
* 文件状态信息。
* 目录项，当前文件指针、文件操作设置等。

**打开文件表**：
* 一个进程一个。
* 一个系统级的。
* 每个卷控制块也保存一个列表。
* 所以如果有文件被打开将不能被加载。

![](./images/figure12-01.png)

**强制和劝告**：
* 强制：根据锁保持情况和需求拒绝访问。
* 劝告：进程可以查找锁的状态来决定怎么做。

## 12.5 文件分配

**如何为一个文件分配数据块**：连续分配、链式分配和索引分配。

**指标**：高效（如何存储利用）和表现（如访问速度）。

**连续分配**：文件头指定起始块和长度。一般底层采用最先匹配，最佳匹配。

它的优势是文件读取表现好、高效的顺序和随机访问；缺点是碎片、文件增长问题。

![](./images/figure12-02.png)

**链式分配**：文件以数据块链表方式存储。文件头包含了到第一块和最后一块的指针。

它的优势是创建、增大、缩小很容易，并且没有碎片；缺点是不可能进行真正的随机访问，而且可靠性比较低（系统突然断电，破话了链表的连续性）。

![](./images/figure12-03.png)

**索引分配**：为每个文件创建一个名为索引数据块的非数据数据块。索引数据块保存文件的片的位置。

它的优势是创建、增大、缩小很容易，没有碎片、支持直接访问；缺点在于当文件很小时，存储索引的开销大，而对于处理大文件可能导致单个索引数据块存储量不足。

![](./images/figure12-04.png)


**对于大文件**可以使用多级分配的方式。


## 12.6 空闲空间列表

空闲空间列表是一种数据结构，用于跟踪在存储（磁盘）中所有未分配的数据块。

用位图代表空闲数据块列表，0表示磁盘块是空闲的。

**不一致性问题**：位图必须保存在硬盘上，在内存的拷贝位图和磁盘上的可能不同。**解决方法**是先在磁盘上置1，再分配块，最后将内存中的对应位置1。

## 12.7 多磁盘管理 - RAID

**分区**：硬件磁盘的一种合适操作系统指定格式的划分。

**卷**：一个拥有一个文件系统实例的可访问的存储空间。

**RAID-0**：将数据均匀的存储在不同的磁盘上，在读取时可以通过并行的方式增加读取速度数据。

**RAID-1**：增加了可靠性，向两个磁盘写入，从任何一个读取（磁盘起到了镜像）。

**RAID-4**：数据均匀分配到n-1个磁盘上，起到RAID-0的作用，然后将n-1个数据对应的奇偶校验值存放在第n个磁盘上，这种情况下允许一个磁盘的损坏。（因为校验和存放在一个磁盘上，每次修写操作都必须修改校验和）

**RAID-5**：在RAID-2的基础上，将奇偶校验值均匀的存放在不同磁盘上。

## 12.8 磁盘调度

**磁盘的性能**：寻道时间（定位到期望的磁道所花费的时间）+ 旋转延迟（从扇区的开始处到达目的处所花费的时间）+ 读取或写入数据的时间。

因为文件可能储存在磁盘的不同位置，不同进程所需要的文件资源可能不同，以至与不同顺序的磁盘请求会导致磁盘转头的不断移动，产生时间开销。

**按顺序请求（FIFS）**会按顺序处理请求，公平对待所有进程，这种调度算法在有很多进程的情况下，接近随机调度的性能。

**最短服务优先（SSTF）**：选择从磁头当前位置需要移动量最少的I/O请求，总是选择最短寻到时间。但是这种调度算法会导致较远的分区的请求产生饥饿情况。

**电梯调度算法（SCAN）**：磁臂在一个方向上移动，满足所有未完成的请求，直到磁臂到达该方向上最后的磁道，然后改变方向。

**循环扫描算法（CSCAN）**：每次磁臂只在一个方向上移动，直到磁臂到达该方向上最后的磁道，然后直接回到起点重新移动。这种算发遵循了公平性的原则。

**N-Step-SCAN**：将磁盘请求队列分成若干长度为N的子队列，磁盘调度将FCFS算法一次处理这些子队列。而每处理一个队列时又是按照SCAN算法。